In [33]:
import sys
sys.path.append('../')

from dotenv import load_dotenv, find_dotenv
envs = load_dotenv(find_dotenv(), override=True)
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [5]:
from archive.weaviate.weaviate_interface import WeaviateClient

In [7]:
endpoint = os.environ['WEAVIATE_ENDPOINT']
api_key = os.environ['WEAVIATE_API_KEY']

In [8]:
client = WeaviateClient(endpoint, api_key)

In [22]:
questions = [
    "How can I optimize my sleep to improve cognitive function and overall well-being?",
    "What are some effective strategies for managing stress and anxiety based on neuroscience?",
    "Can you explain the science behind meditation and its benefits for mental health?",
    "How does nutrition impact brain health and cognitive performance?",
    "What are the best methods for enhancing focus and concentration?",
    "Can you discuss the relationship between exercise and brain function?",
    "How does exposure to natural light affect sleep quality and circadian rhythms?",
    "What are the most effective techniques for improving memory and learning?",
    "Can you explain the role of neurotransmitters in regulating mood and behavior?",
    "How does chronic stress impact brain structure and function?",
    "What are the implications of neuroplasticity for personal growth and development?",
    "How can we use breathing techniques to modulate our nervous system and reduce stress?",
    "Can you discuss the science of motivation and goal-setting from a neuroscience perspective?",
    "What are some effective ways to optimize brain health as we age?",
    "How does technology use affect brain function and mental well-being?",
    "Can you explain the effects of different types of music on the brain and mood?",
    "What are the benefits of exposure to nature for mental health and cognitive function?",
    "How can we cultivate resilience and adaptability in the face of challenges?",
    "Can you discuss the relationship between gut health and mental health?",
    "What are some practical strategies for improving emotional regulation and self-control?"
  ]

In [23]:
def main(query: str, collection_name: str, query_fields: list[str]=['content', 'title']):
    return client.hybrid_search(query, collection_name, query_fields, display_properties=['content'])

### Single Query: v3

In [66]:
%%time
answer_v3 = client.hybrid_search("What is serotonin good for", class_name, display_properties=['content'])

CPU times: user 1.36 s, sys: 32.2 ms, total: 1.39 s
Wall time: 386 ms


### Multi-Query (n=20) : v3

In [67]:
%%time
answers_v3 = []
for q in questions:
    answers_v3.append(client.hybrid_search(qu, class_name, display_properties=['content']))

CPU times: user 26.6 s, sys: 502 ms, total: 27.1 s
Wall time: 4.95 s


### Python Multithreading (n=20) : v3

In [69]:
%%time
results = []
with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as exec:
    futures = [exec.submit(client.hybrid_search, q, class_name, \
                           display_properties=['title', 'content']) for q in questions]
    for future in as_completed(futures):
        results.append(future.result())

CPU times: user 996 ms, sys: 4.59 s, total: 5.59 s
Wall time: 1.02 s
